# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [2]:
# import the train dataset ( 1000 first rows)
import pandas as pd
df = pd.read_csv("C:\\Users\\Joe\\Desktop\\py\\workflow\\train.csv", nrows=1000,parse_dates=['pickup_datetime'])

In [3]:
100 * df.isnull().sum().sort_values(ascending=False)/len(df)


key                  0.0
fare_amount          0.0
pickup_datetime      0.0
pickup_longitude     0.0
pickup_latitude      0.0
dropoff_longitude    0.0
dropoff_latitude     0.0
passenger_count      0.0
dtype: float64

In [4]:
size_before = len(df)
df = df.drop_duplicates()
size_after = len(df)
print(str(size_before - size_after) + " duplicates were removed.")

0 duplicates were removed.


In [6]:
def clean_data(df, test=False, predict=False):
    df = df.drop(["key"], axis=1)
    df = df.dropna(how='any', axis='rows')
    df = df[(df.dropoff_latitude != 0) | (df.dropoff_longitude != 0)]
    df = df[(df.pickup_latitude != 0) | (df.pickup_longitude != 0)]
    if "fare_amount" in list(df):
        df = df[df.fare_amount.between(0, 4000)]
    df = df[df.passenger_count < 8]
    df = df[df.passenger_count >= 0]
    df = df[df["pickup_latitude"].between(left=40, right=42)]
    df = df[df["pickup_longitude"].between(left=-74.3, right=-72.9)]
    df = df[df["dropoff_latitude"].between(left=40, right=42)]
    df = df[df["dropoff_longitude"].between(left=-74, right=-72.9)]
    return df
data_train = clean_data(df)
data_train.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


In [7]:
# prepare X and y
X_train = data_train.drop(["fare_amount"], axis=1)
y_train = data_train["fare_amount"]

In [52]:
X = X_train
y = y_train

In [53]:
# Hold out ( train and test split )
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
X_train.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [9]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [14]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):
        # A COMPPLETER
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
    
        return pd.DataFrame(haversine_vectorized(X)).rename(columns={0: "course distance [km]"}).copy()

In [15]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,course distance [km]
0,1.030764
1,8.450134
2,1.389525
3,2.799270
4,1.999157


In [16]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self, time_column, time_zone_name='America/New_York'):
       # A COMPLETER
                self.time_column = time_column
                self.time_zone_name = time_zone_name
    def extract_time_features(self, X):
        timezone_name = self.time_zone_name
        time_column = self.time_column
        df = X.copy()
        df.index = pd.to_datetime(df[time_column])
        df.index = df.index.tz_convert(timezone_name)
        df["dow"] = df.index.weekday
        df["hour"] = df.index.hour
        df["month"] = df.index.month
        df["year"] = df.index.year        
        return df
    def fit(self, X, y=None):
        # A COMPLETER
        return self
    def transform(self, X, y=None):
        # A COMPLETER 
        return self.extract_time_features(X)[['dow', 'hour', 'month', 'year']].reset_index(drop=True)

In [17]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour,month,year
0,0,13,6,2009
1,1,11,1,2010
2,2,20,8,2011
3,5,0,4,2012
4,1,2,3,2010


###  Prepocessing pipeline

In [18]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [38]:

# create distance pipeline dist_pipe
dist_pipe = Pipeline([
    ('distance', DistanceTransformer()),
    ('distance scale',StandardScaler() )
])

# display distance pipeline
dist_pipe

Pipeline(steps=[('distance', DistanceTransformer()),
                ('distance scale', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# create time pipeline time_pipe

pipeTime = Pipeline([
    ('time_features_create', TimeFeaturesEncoder('pickup_datetime')),
    ('time_features_ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])
# display time pipeline
pipeTime

Pipeline(steps=[('time_features_create',
                 TimeFeaturesEncoder(time_column='pickup_datetime')),
                ('time_features_ohe',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler


In [40]:
preprocessor = ColumnTransformer([
    ('distance',dist_pipe, ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']),
    ('time', pipeTime, ['pickup_datetime'])
])
preprocessor

ColumnTransformer(transformers=[('distance',
                                 Pipeline(steps=[('distance',
                                                  DistanceTransformer()),
                                                 ('distance scale',
                                                  StandardScaler())]),
                                 ['pickup_longitude', 'pickup_latitude',
                                  'dropoff_longitude', 'dropoff_latitude']),
                                ('time',
                                 Pipeline(steps=[('time_features_create',
                                                  TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                 ('time_features_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['pickup_datetime'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [42]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe
pipe = Pipeline([
    ('dist_and_time', preprocessor),
    ('regLinear', LinearRegression())
])
# display the pipeline with model
pipe

Pipeline(steps=[('dist_and_time',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('distance',
                                                                   DistanceTransformer()),
                                                                  ('distance '
                                                                   'scale',
                                                                   StandardScaler())]),
                                                  ['pickup_longitude',
                                                   'pickup_latitude',
                                                   'dropoff_longitude',
                                                   'dropoff_latitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_features_create',
                                                                   TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                                  ('time_features_ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['pickup_datetime'])])),
                ('regLinear', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [61]:
# train the pipelined model
pipe.fit(X_train, y_train)

# compute y_pred on the test set
y_pred = pipe.predict(X_test)

Use the RMSE to evaluate the performance of the model:

In [62]:
y_pred

array([15.10302734,  8.08349609,  9.93457031,  9.40771484, 13.69873047,
        5.76269531, 35.18261719, 10.39160156,  5.17089844, 52.62890625,
        7.99072266,  5.61914062, 11.1328125 , 31.03027344,  4.07080078,
        7.91943359,  4.51904297, 15.46435547,  6.79638672,  6.00097656,
        7.09277344, 13.66162109,  5.19970703,  0.87744141,  5.94970703,
        2.14550781,  3.91699219,  7.42480469, 11.50537109,  6.9609375 ,
        3.12646484, 36.1796875 ,  9.93115234,  4.26367188,  6.26220703,
        9.47607422, 10.45410156, 11.90771484,  6.98583984, 14.82324219,
        8.76953125, 12.43994141,  3.17236328,  3.92431641,  8.02246094,
        6.57177734, 25.72607422,  3.74853516,  9.58300781, 15.65576172,
        8.20800781, 18.98193359,  8.08789062,  8.75927734, 18.67626953,
        5.39111328,  9.20654297, 14.72216797, 11.86132812,  9.56542969,
        6.68652344,  9.63476562, 16.89404297,  4.75537109,  6.83007812,
        5.85107422, 10.48144531, 25.12304688,  9.97900391, 11.52

In [63]:
import numpy as np
def compute_rmse(y_test, y_pred):
    
    return np.sqrt(np.mean(np.square(y_test - y_pred)))

In [59]:
compute_rmse(y_pred, y_test)

nan

In [64]:
compute_rmse(y_pred, y_test)

4.830248313694628

In [19]:
# call compute_rmse
compute_rmse(y_pred, y_test)

10.175329127383847

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [65]:
# implement get_data() function
def get_data(nrows=10000):
    '''returns a DataFrame with nrows from s3 bucket'''
    df = pd.read_csv("C:\\Users\\Joe\\Desktop\\py\\workflow\\train.csv", nrows=1000,parse_dates=['pickup_datetime'])
    return df

In [66]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    df = df.drop(["key"], axis=1)
    df = df.dropna(how='any', axis='rows')
    df = df[(df.dropoff_latitude != 0) | (df.dropoff_longitude != 0)]
    df = df[(df.pickup_latitude != 0) | (df.pickup_longitude != 0)]
    if "fare_amount" in list(df):
        df = df[df.fare_amount.between(0, 4000)]
    df = df[df.passenger_count < 8]
    df = df[df.passenger_count >= 0]
    df = df[df["pickup_latitude"].between(left=40, right=42)]
    df = df[df["pickup_longitude"].between(left=-74.3, right=-72.9)]
    df = df[df["dropoff_latitude"].between(left=40, right=42)]
    df = df[df["dropoff_longitude"].between(left=-74, right=-72.9)]
    return df

In [68]:
# implement set_pipeline() function
def set_pipeline():
    pipe = Pipeline([
        ('dist_and_time', preprocessor),
        ('regLinear', LinearRegression())
    ])
    return pipe

In [69]:
# implement train() function
def train(X_train, y_train, pipe):
    pipe.fit(X_train, y_train)
    return pipeline

In [70]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    y_pred = pipe.predict(X_test)
    rmse = np.sqrt(np.mean(np.square(y_test - y_pred)))
    print(rmse)
    return rmse

In [73]:
evaluate(X_test, y_test, pipe)

4.830248313694628


4.830248313694628

### Test the complete worflow

Use the above functions to test the complete workflow.

In [74]:
# store the data in a DataFrame
df = get_data()

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

10.49131441008365


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀